# **Heart disease prediction model**

# **01_data_preprocessing**

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load dataset
df = pd.read_csv(r"C:\Users\fadya\Downloads\heart.csv")
print("Loaded rows:", len(df))
print("Columns:", df.columns.tolist())
print("Missing per column:\n", df.isnull().sum())

# Basic EDA
df.hist(figsize=(15, 12), bins=20, edgecolor='black')
plt.suptitle("Histograms of Features", fontsize=16)
plt.show()

plt.figure(figsize=(12, 8))
corr = df.corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap", fontsize=16)
plt.show()

# Separate target
target = df['target']
df = df.drop('target', axis=1)

# Define numeric and categorical columns
num_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca']
cat_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

# Scaling numeric columns
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Column Transformer for future pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), num_cols),
        ('cat', OneHotEncoder(), cat_cols)
    ]
)

# Boxplots
plt.figure(figsize=(15, 10))
df.boxplot()
plt.title("Boxplots of Features", fontsize=16)
plt.xticks(rotation=45)
plt.show()

# **02_pca_analysis**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Assuming df and target loaded from preprocessing
pca = PCA()
X_pca = pca.fit_transform(df)
explained_var = pca.explained_variance_ratio_

# Cumulative explained variance
cumulative_var = np.cumsum(explained_var)
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(cumulative_var) + 1), cumulative_var, marker='o', linestyle='--')
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("Explained Variance vs Number of Components")
plt.grid(True)
plt.axhline(y=0.95, color='gray', linestyle=':', linewidth=1)
plt.show()

# PCA scatter plot (first two components)
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=target, cmap='coolwarm', edgecolor='k', alpha=0.7)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA - First Two Components")
plt.colorbar(label="Target")
plt.show()

# **03_feature_selection**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, SelectKBest, chi2

# Fit Random Forest for RFE
rf = RandomForestClassifier(random_state=42)
rf.fit(df, target)

rfe_selector = RFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=8)
rfe_selector.fit(df, target)
rfe_features = df.columns[rfe_selector.support_]
print("Selected by RFE:", list(rfe_features))

# Chi-Square selection
chi2_selector = SelectKBest(score_func=chi2, k=8)
chi2_selector.fit(df, target)
chi2_features = df.columns[chi2_selector.get_support()]
print("Selected by Chi-Square:", list(chi2_features))

# **04_supervised_learning**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, classification_report
import matplotlib.pyplot as plt

# Split data
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=42, stratify=target)

# Models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

results = {}
plt.figure(figsize=(8, 6))

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    results[name] = [accuracy_score(y_test, y_pred),
                     precision_score(y_test, y_pred),
                     recall_score(y_test, y_pred),
                     f1_score(y_test, y_pred)]
    
    print(f"\n{name} Classification Report:\n", classification_report(y_test, y_pred))
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label=f"{name} (AUC = {auc(fpr, tpr):.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend(loc="lower right")
plt.show()

# **05_unsupervised_learning**

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt

# KMeans Elbow
inertia = []
K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K, inertia, marker="o", linestyle="--")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Inertia")
plt.title("Elbow Method for Optimal K")
plt.show()

# Fit KMeans with K=2
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
clusters_kmeans = kmeans.fit_predict(df)
print("Adjusted Rand Index (KMeans vs Target):", adjusted_rand_score(target, clusters_kmeans))

# Hierarchical Clustering
plt.figure(figsize=(12, 6))
Z = linkage(df, method="ward")
dendrogram(Z, truncate_mode="lastp", p=30, leaf_rotation=45, leaf_font_size=10)
plt.title("Hierarchical Clustering Dendrogram")
plt.show()

hc = AgglomerativeClustering(n_clusters=2, metric="euclidean", linkage="ward")
clusters_hc = hc.fit_predict(df)
print("Adjusted Rand Index (Hierarchical vs Target):", adjusted_rand_score(target, clusters_hc))

# **06_hyperparameter_tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Random Forest
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10, 20], 'min_samples_split': [2, 5, 10]}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5, scoring='accuracy', n_jobs=-1)
rf_grid.fit(X_train, y_train)
print("Best RF:", rf_grid.best_params_, rf_grid.best_score_)

# SVM
svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}
svm_random = RandomizedSearchCV(SVC(probability=True, random_state=42), svm_params, n_iter=5, cv=5,
                                scoring='accuracy', n_jobs=-1, random_state=42)
svm_random.fit(X_train, y_train)
print("Best SVM:", svm_random.best_params_, svm_random.best_score_)

# Logistic Regression
log_params = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
log_grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), log_params, cv=5, scoring='accuracy', n_jobs=-1)
log_grid.fit(X_train, y_train)
print("Best LR:", log_grid.best_params_, log_grid.best_score_)

# Compare optimized models
optimized_models = {
    "Random Forest Optimized": rf_grid.best_estimator_,
    "SVM Optimized": svm_random.best_estimator_,
    "Logistic Regression Optimized": log_grid.best_estimator_
}

opt_results = {}
for name, model in optimized_models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    opt_results[name] = acc
    print(f"\n{name} Classification Report:\n", classification_report(y_test, y_pred))

opt_results_df = pd.DataFrame(opt_results, index=["Accuracy"])
print(opt_results_df)

# **07_models/final_pipeline**

In [ ]:
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Load processed data
df = pd.read_csv("data/heart_disease_processed.csv")  # or processed df from notebooks
target = df['target']
X = df.drop('target', axis=1)
y = target

# Define numeric and categorical columns
num_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca']
cat_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), num_cols),
        ('cat', OneHotEncoder(), cat_cols)
    ]
)

# Final Pipeline
final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_estimators=200, max_depth=6, random_state=42))
])

# Fit pipeline
final_pipeline.fit(X, y)

# Save pipeline
joblib.dump(final_pipeline, "models/best_model.pkl")
print("Pipeline saved to models/best_model.pkl")

# **08_ui/app**

In [ ]:
import streamlit as st
import pandas as pd
import joblib

# Load saved pipeline
model = joblib.load("models/best_model.pkl")

st.title("Heart Disease Prediction App")

# User inputs
age = st.number_input("Age", min_value=1, max_value=120, step=1)
sex = st.selectbox("Sex", ["male", "female"])
cp = st.selectbox("Chest Pain Type (cp)", [0, 1, 2, 3])
trestbps = st.number_input("Resting Blood Pressure", min_value=50, max_value=250, step=1)
chol = st.number_input("Cholesterol", min_value=100, max_value=600, step=1)
fbs = st.selectbox("Fasting Blood Sugar > 120 mg/dl", [0, 1])
restecg = st.selectbox("Resting ECG", [0, 1, 2])
thalach = st.number_input("Maximum Heart Rate", min_value=50, max_value=250, step=1)
exang = st.selectbox("Exercise Induced Angina", [0, 1])
oldpeak = st.number_input("ST Depression", min_value=0.0, max_value=10.0, step=0.1)
slope = st.selectbox("Slope", [0, 1, 2])
ca = st.selectbox("Number of Major Vessels (ca)", [0, 1, 2, 3, 4])
thal = st.selectbox("Thalassemia", [0, 1, 2, 3])

# Convert inputs to a dataframe
input_data = pd.DataFrame([[age, 1 if sex=="male" else 0, cp, trestbps, chol, fbs,
                            restecg, thalach, exang, oldpeak, slope, ca, thal]],
                          columns=["age","sex","cp","trestbps","chol","fbs","restecg",
                                   "thalach","exang","oldpeak","slope","ca","thal"])

# Prediction button
if st.button("Predict"):
    pred = model.predict(input_data)
    if pred[0] == 1:
        st.error("You may have heart disease!")
    else:
        st.success("You are healthy!")

# Optional: Show prediction probability
if st.checkbox("Show Prediction Probability"):
    pred_prob = model.predict_proba(input_data)
    st.write("Probability of Heart Disease:", round(pred_prob[0][1]*100, 2), "%")